<a href="https://colab.research.google.com/github/azzindani/03_LLM/blob/main/00_LLM_Fine_Tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Modules

In [1]:
!pip install --upgrade transformers
!pip install peft
!pip install -U bitsandbytes
!pip install datasets
!pip install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 32.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import pathlib
import torch
import numpy as np

from datasets import load_dataset
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from trl import SFTTrainer

from transformers import (
  AutoTokenizer,
  AutoModelForCausalLM,
  AutoModel,
  AutoModelForSequenceClassification,
  DataCollatorForLanguageModeling,
  Trainer,
  TrainingArguments,
  pipeline,
  TextDataset,
  EvalPrediction,
  DataCollatorWithPadding,
  GenerationConfig,
  BitsAndBytesConfig
)

from peft import (
  LoraConfig,
  PeftModelForSequenceClassification,
  TaskType,
  AutoPeftModelForSequenceClassification,
  get_peft_model,
  prepare_model_for_kbit_training
)

if torch.cuda.is_available():
  print("GPU is available!")
else:
  print("GPU is not available.")

GPU is available!


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Import Model

In [4]:
#url = 'https://huggingface.co/Qwen/Qwen2.5-0.5B'
#model_name = url.split('.co/')[-1]

model_name = 'Qwen/Qwen2.5-0.5B-Instruct'

In [5]:
'''bnb_config = BitsAndBytesConfig(
  load_in_4bit = True,
  bnb_4bit_quant_type = 'nf4',
  bnb_4bit_compute_dtype = torch.float16,
  bnb_4bit_use_double_quant = True,
)

model = AutoModelForCausalLM.from_pretrained(
  model_path,
  quantization_config = bnb_config,
  trust_remote_code = True
)#.to(device) #'''

"bnb_config = BitsAndBytesConfig(\n  load_in_4bit = True,\n  bnb_4bit_quant_type = 'nf4',\n  bnb_4bit_compute_dtype = torch.float16,\n  bnb_4bit_use_double_quant = True,\n)\n\nmodel = AutoModelForCausalLM.from_pretrained(\n  model_path,\n  quantization_config = bnb_config,\n  trust_remote_code = True\n)#.to(device) #"

In [6]:
model = AutoModelForCausalLM.from_pretrained(
  model_name,
  torch_dtype = torch.float16,
  trust_remote_code = True
)#.to(device) #'''

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [7]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((

In [8]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

Total parameters : 494032768
Trainable parameters : 494032768
Trainable percentage: 100.00%


## Import Tokenizer

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [10]:
tokenizer

Qwen2TokenizerFast(name_or_path='Qwen/Qwen2.5-0.5B-Instruct', vocab_size=151643, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|im_end|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, single_word=False, 

## Import Dataset

In [11]:
def preprocess(example):
  example['prompt'] = ' '.join([example[feature] for feature in features])
  return example

def preprocess_function(examples):
  inputs = tokenizer(examples['instruction'], padding = 'max_length', truncation = True)
  inputs['labels'] = inputs['input_ids'].copy()
  return inputs

def tokenize_datasets(dataset):
  tokenized_dataset = dataset.map(
    lambda example: tokenizer(
      example['prompt'],
      truncation = True,
      max_length = 128,
      ),
    batched = True,
    remove_columns = ['prompt']
  )
  return tokenized_dataset

In [12]:
#url = 'https://huggingface.co/datasets/KingNish/reasoning-base-20k'
#dataset_name = url.split('datasets/')[-1]

dataset_name = 'migtissera/Synthia-v1.5-II'

In [13]:
dataset = load_dataset(dataset_name)
dataset

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Synthia-v1.5-3.jsonl:   0%|          | 0.00/11.9M [00:00<?, ?B/s]

Synthia-v1.5-4.jsonl:   0%|          | 0.00/154M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/21819 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['system', 'instruction', 'response'],
        num_rows: 21819
    })
})

In [14]:
dataset['train'].select(range(5)).to_pandas().head()

,system,instruction,response
0,You are an AI assistant. Provide a detailed an...,"Here is a detailed, multi-part question on a s...",Here is a detailed response to your multi-part...
1,\n Answer the Question by exploring multipl...,Here is a potential multi-part question on a s...,Here is my attempt at answering this complex m...
2,You are an AI assistant. User will you give yo...,Here is a multi-part graduate level question o...,Here is my analysis of the graduate-level cybe...
3,You are an AI assistant that follows instructi...,Here is a multi-part question on the topic of ...,This is a great question that delves into some...
4,You are an AI assistant that helps people find...,"Here is a detailed, multi-part question on an ...","Thank you for this excellent, comprehensive qu..."


In [15]:
features = list(dataset['train'].features.keys())
print(features)

['system', 'instruction', 'response']


In [16]:
dataset = dataset.map(preprocess, remove_columns = features)
dataset

Map:   0%|          | 0/21819 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt'],
        num_rows: 21819
    })
})

In [17]:
dataset = dataset['train'].train_test_split(test_size = 0.1, seed = 42)
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt'],
        num_rows: 19637
    })
    test: Dataset({
        features: ['prompt'],
        num_rows: 2182
    })
})

In [18]:
train_dataset = dataset['train']
test_dataset = dataset['test']
train_dataset

Dataset({
    features: ['prompt'],
    num_rows: 19637
})

In [19]:
train_dataset.select(range(5)).to_pandas().head()

,prompt
0,"Here is a detailed, multi-part question on an..."
1,"Here is a detailed, multi-part question on an..."
2,"Here is a detailed, multi-part question on a ..."
3,Here is a graduate level multi-part question ...
4,"Here is a detailed, multi-part question on a ..."


In [20]:
train_dataset = tokenize_datasets(train_dataset)
test_dataset = tokenize_datasets(test_dataset)
train_dataset

Map:   0%|          | 0/19637 [00:00<?, ? examples/s]

Map:   0%|          | 0/2182 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 19637
})

In [21]:
train_dataset.select(range(5)).to_pandas().head()

,input_ids,attention_mask
0,"[5692, 374, 264, 11682, 11, 7299, 28037, 3405,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"[5692, 374, 264, 11682, 11, 7299, 28037, 3405,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"[5692, 374, 264, 11682, 11, 7299, 28037, 3405,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"[5692, 374, 264, 19023, 2188, 7299, 28037, 340...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"[5692, 374, 264, 11682, 11, 7299, 28037, 3405,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


## Training Set Up

In [22]:
#data_collator = DataCollatorWithPadding(tokenizer = tokenizer)
data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm = False)

In [23]:
def compute_metrics(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis = 1)
  precision, recall, f1, _ = precision_recall_fscore_support(
    p.label_ids,
    preds,
    average = 'weighted'
  )
  matrix = {
    'accuracy': accuracy_score(p.label_ids, preds),
    'f1': f1, 'precision': precision,
    'recall': recall
  }
  return matrix

In [24]:
model.config.pad_token_id = tokenizer.pad_token_id

In [25]:
torch.cuda.empty_cache()

## Training Model

In [26]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((

In [27]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

Total parameters : 494032768
Trainable parameters : 494032768
Trainable percentage: 100.00%


In [28]:
save_path = './model'
batch = 16
epochs = 3

training_args = TrainingArguments(
  output_dir = save_path,
  evaluation_strategy = 'no',
  learning_rate = 2e-5,
  per_device_train_batch_size = batch,
  num_train_epochs = epochs,
  weight_decay = 0.01,
  logging_dir = './logs',
  save_strategy = 'epoch',
  logging_steps = 100,
  warmup_ratio = 0.1,
  #fp16=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [29]:
trainer = Trainer(
  model = model,
  args = training_args,
  train_dataset = train_dataset.select(range(10000)),
  eval_dataset = test_dataset.select(range(1000)),
  compute_metrics = compute_metrics,
  tokenizer = tokenizer,
  data_collator = data_collator,
)
trainer

In [30]:
trainer.train()

Step,Training Loss
100,1.132900
200,0.000000
300,0.000000
400,0.000000
500,0.000000
600,0.000000
700,0.000000
800,0.000000
900,0.000000
1000,0.000000


TrainOutput(global_step=1875, training_loss=0.06041907958984375, metrics={'train_runtime': 1045.9248, 'train_samples_per_second': 28.683, 'train_steps_per_second': 1.793, 'total_flos': 8245972500480000.0, 'train_loss': 0.06041907958984375, 'epoch': 3.0})

## Model Evaluation

In [31]:
#evaluation_results = trainer.evaluate()
#print('Evaluation Results:', evaluation_results)

## Save Model

In [32]:
save_model = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model
#save_model.save_pretrained(save_path)